In [ ]:
import glob

In [ ]:
good_images = glob.glob("/root/data/priority_queue/frames-resized/good/*.jpg")
print("Number of good images {}".format(len(good_images)))
bad_images = glob.glob("/root/data/priority_queue/frames-resized/blurry/*.jpg")
bad_images += glob.glob("/root/data/priority_queue/frames-resized/dark/*.jpg")
print("Number of bad images {}".format(len(bad_images)))

Load some annotations

In [ ]:
import json
import os

In [ ]:
# consensus crops -> frames
image_classes = json.load(open("/root/data/priority_queue/frames/image_classes.json"))

In [ ]:
# for (k,v) in image_classes.items():
#     print(v)
#     print([int(k) for k in v])
#     break

In [ ]:
for img in bad_images:
    name = os.path.basename(img)
    if name not in image_classes:
        print("red alert")

Create generator

Experiences:
* BW classifier
* RGB classifer: overfitting done / 
* split per class with sigmoid head

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [ ]:
import random
random.seed(124)
random.shuffle(bad_images)
random.shuffle(good_images)

In [ ]:
cutoff_good = int(len(good_images)*0.8)
cutoff_bad = int(len(bad_images)*0.8)

train_good = good_images[:cutoff_good]
val_good = good_images[cutoff_good:]

train_bad = bad_images[:cutoff_bad]
val_bad = bad_images[cutoff_bad:]

In [ ]:
ngpus = 2
new_shape = (224, 224)
batch_size = 32*ngpus
classes = 3

In [ ]:
import keras

In [ ]:
from imgaug import augmenters as iaa

In [ ]:
augmenters = {1: iaa.GaussianBlur((0.0, 3.0), name="GaussianBlur"),
              2: iaa.Add((-100, 0))}
sometimes = lambda aug: iaa.Sometimes(0.5, aug)

In [ ]:
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, good_images, bad_images, batch_size=batch_size, dim=(224, 224, 3)):
        'Initialization'
        self.good_images = good_images
        self.batch_size = batch_size
        self.bad_images = bad_images
        self.dim = dim
        self.on_epoch_end()
        
    def __len__(self):
        'Denotes the number of batches per epoch'
#         return int((len(self.good_images) + len(self.bad_images)) / self.batch_size)
        return (np.min([len(self.good_images), len(self.bad_images)])) // (self.batch_size)
    
    def __getitem__(self, index):
        'Generate one batch of data'
        xbatch = []
        ybatch = []
        start = index * self.batch_size
        end = (index+1)*self.batch_size
#         print(start, end)
        for i in range(start, end, 1):
            auglist = [iaa.Fliplr(0.5)]
            k = i // 2
            if i % 2 == 0:
                path = self.good_images[k]
                label = [1, 0, 0]
            else:
                k = i // 2 # - 1
                path = self.bad_images[k]
                label = [0] + image_classes[os.path.basename(path)]
#                 auglist += [sometimes(iaa.GaussianBlur((0.0, 3.0))), 
#                             sometimes(iaa.Add((-100, 0)))]
                
            seq = iaa.Sequential(auglist)    
            image = cv2.imread(path)
            image = cv2.resize(image, new_shape)
            image = seq.augment_image(image)
            xbatch.append(image)
            ybatch.append(label)

        return np.array(xbatch), np.array(ybatch)
    
    def on_epoch_end(self):
        random.shuffle(self.good_images)
        random.shuffle(self.bad_images)

In [ ]:
traingen = DataGenerator(train_good, train_bad)
valgen = DataGenerator(val_good, val_bad)

In [ ]:
xb, yb = traingen[0]

In [ ]:
print(xb.shape, yb.shape)

In [ ]:
# for i in range(batch_size):
#     plt.imshow(xb[i, ...])
#     plt.show()
#     print(yb[i])

load and compile model

In [ ]:
import os

from keras import layers
from keras.models import Model
from keras.applications.resnet50 import ResNet50
from keras.applications.mobilenet import MobileNet
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.optimizers import Adam
from keras.utils import multi_gpu_model
import tensorflow as tf

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0, 1"

Models:
* resnet50 = overfit
* mobilenet 
    

In [ ]:
# body = ResNet50(include_top=False, 
#                  weights='imagenet',
#                  input_shape=(256, 256, 3),
#                  pooling="avg")
# x = body.output
# x = layers.Dense(1, activation='sigmoid', name='fc1000')(x)

In [ ]:
# with tf.device('/cpu:0'):
#     alpha=1.0
#     shape = (1, 1, int(1024 * alpha))
#     dropout=1e-3
#     body = MobileNet(include_top=False,
#                      weights='imagenet',
#                      input_shape=(224, 224, 3),
#                      pooling=None
#                     )
    
#     x = layers.GlobalAveragePooling2D()(body.output)
#     x = layers.Reshape(shape, name='reshape_1')(x)
#     x = layers.Dropout(dropout, name='dropout')(x)
#     x = layers.Conv2D(classes, (1, 1),
#                       padding='same',
#                       name='conv_preds')(x)
#     x = layers.Activation('sigmoid', name='act_sigmoid')(x)
#     x = layers.Reshape((classes,), name='reshape_2')(x)
    
#     single_model = Model([body.input], [x])

In [ ]:
# with tf.device('/cpu:0'):
#     alpha = 1.0
#     dropout = 1e-2
#     body = MobileNetV2(include_top=False,
#                      weights='imagenet',
#                      input_shape=(224, 224, 3),
#                      pooling=None
#                     )
    
#     x = layers.GlobalAveragePooling2D()(body.output)
#     x = layers.Dense(classes, activation='sigmoid',
#                      use_bias=True, name='Logits')(x)
    
#     single_model = Model([body.input], [x])

In [ ]:
import keras.backend as K

In [ ]:
def _make_divisible(v, divisor, min_value=None):
    if min_value is None:
        min_value = divisor
    new_v = max(min_value, int(v + divisor / 2) // divisor * divisor)
    # Make sure that round down does not go down by more than 10%.
    if new_v < 0.9 * v:
        new_v += divisor
    return new_v

def _inverted_res_block(inputs, expansion, stride, alpha, filters, block_id):
    in_channels = K.int_shape(inputs)[-1]
    pointwise_conv_filters = int(filters * alpha)
    pointwise_filters = _make_divisible(pointwise_conv_filters, 8)
    x = inputs
    prefix = 'block_{}_'.format(block_id)

    if block_id:
        # Expand
        x = layers.Conv2D(expansion * in_channels,
                          kernel_size=1,
                          padding='same',
                          use_bias=False,
                          activation=None,
                          name=prefix + 'expand')(x)
        x = layers.BatchNormalization(epsilon=1e-3,
                                      momentum=0.999,
                                      name=prefix + 'expand_BN')(x)
        x = layers.ReLU(6., name=prefix + 'expand_relu')(x)
    else:
        prefix = 'expanded_conv_'

    # Depthwise
    if stride == 2:
        x = layers.ZeroPadding2D(padding=correct_pad(K, x, 3),
                                 name=prefix + 'pad')(x)
    x = layers.DepthwiseConv2D(kernel_size=3,
                               strides=stride,
                               activation=None,
                               use_bias=False,
                               padding='same' if stride == 1 else 'valid',
                               name=prefix + 'depthwise')(x)
    x = layers.BatchNormalization(epsilon=1e-3,
                                  momentum=0.999,
                                  name=prefix + 'depthwise_BN')(x)

    x = layers.ReLU(6., name=prefix + 'depthwise_relu')(x)

    # Project
    x = layers.Conv2D(pointwise_filters,
                      kernel_size=1,
                      padding='same',
                      use_bias=False,
                      activation=None,
                      name=prefix + 'project')(x)
    x = layers.BatchNormalization(
        epsilon=1e-3, momentum=0.999, name=prefix + 'project_BN')(x)

    if in_channels == pointwise_filters and stride == 1:
        return layers.Add(name=prefix + 'add')([inputs, x])
    return x

def correct_pad(backend, inputs, kernel_size):
    """Returns a tuple for zero-padding for 2D convolution with downsampling.
    # Arguments
        input_size: An integer or tuple/list of 2 integers.
        kernel_size: An integer or tuple/list of 2 integers.
    # Returns
        A tuple.
    """
    img_dim = 2 if backend.image_data_format() == 'channels_first' else 1
    input_size = backend.int_shape(inputs)[img_dim:(img_dim + 2)]

    if isinstance(kernel_size, int):
        kernel_size = (kernel_size, kernel_size)

    if input_size[0] is None:
        adjust = (1, 1)
    else:
        adjust = (1 - input_size[0] % 2, 1 - input_size[1] % 2)

    correct = (kernel_size[0] // 2, kernel_size[1] // 2)

    return ((correct[0] - adjust[0], correct[0]),
            (correct[1] - adjust[1], correct[1]))

with tf.device('/cpu:0'):
    alpha=1.0
    dropout=5e-3
    depth_multiplier=1.0

    img_input = layers.Input(shape=(224, 224, 3))
    first_block_filters = _make_divisible(32 * alpha, 8)
    x = layers.ZeroPadding2D(padding=correct_pad(K, img_input, 3),
                             name='Conv1_pad')(img_input)
    x = layers.Conv2D(first_block_filters,
                      kernel_size=3,
                      strides=(2, 2),
                      padding='valid',
                      use_bias=False,
                      name='Conv1')(x)
    x = layers.BatchNormalization(
        epsilon=1e-3, momentum=0.999, name='bn_Conv1')(x)
    x = layers.ReLU(6., name='Conv1_relu')(x)

    x = _inverted_res_block(x, filters=16, alpha=alpha, stride=1,
                            expansion=1, block_id=0)

    x = _inverted_res_block(x, filters=24, alpha=alpha, stride=2,
                            expansion=6, block_id=1)
    x = _inverted_res_block(x, filters=24, alpha=alpha, stride=1,
                            expansion=6, block_id=2)

    x = _inverted_res_block(x, filters=32, alpha=alpha, stride=2,
                            expansion=6, block_id=3)
    x = _inverted_res_block(x, filters=32, alpha=alpha, stride=1,
                            expansion=6, block_id=4)
    x = _inverted_res_block(x, filters=32, alpha=alpha, stride=1,
                            expansion=6, block_id=5)

    x = _inverted_res_block(x, filters=64, alpha=alpha, stride=2,
                            expansion=6, block_id=6)
    x = _inverted_res_block(x, filters=64, alpha=alpha, stride=1,
                            expansion=6, block_id=7)
    x = _inverted_res_block(x, filters=64, alpha=alpha, stride=1,
                            expansion=6, block_id=8)
    x = _inverted_res_block(x, filters=64, alpha=alpha, stride=1,
                            expansion=6, block_id=9)

    x = _inverted_res_block(x, filters=96, alpha=alpha, stride=1,
                            expansion=6, block_id=10)
    x = _inverted_res_block(x, filters=96, alpha=alpha, stride=1,
                            expansion=6, block_id=11)
    x = _inverted_res_block(x, filters=96, alpha=alpha, stride=1,
                            expansion=6, block_id=12)

#     x = _inverted_res_block(x, filters=160, alpha=alpha, stride=2,
#                             expansion=6, block_id=13)
#     x = _inverted_res_block(x, filters=160, alpha=alpha, stride=1,
#                             expansion=6, block_id=14)
#     x = _inverted_res_block(x, filters=160, alpha=alpha, stride=1,
#                             expansion=6, block_id=15)

#     x = _inverted_res_block(x, filters=320, alpha=alpha, stride=1,
#                             expansion=6, block_id=16)
    
    last_block_filters = 1280
    x = layers.Conv2D(last_block_filters,
                      kernel_size=1,
                      use_bias=False,
                      name='Conv_1')(x)
    x = layers.BatchNormalization(epsilon=1e-3,
                                  momentum=0.999,
                                  name='Conv_1_bn')(x)
    x = layers.ReLU(6., name='out_relu')(x)
    
    # top
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(classes, activation='sigmoid',
                     use_bias=True, name='Logits')(x)
    
    single_model = Model([img_input], [x])


In [ ]:
model = multi_gpu_model(single_model, gpus=ngpus)

In [ ]:
initial_lrate = 1e-3
adam = Adam(lr=initial_lrate)

In [ ]:
import keras.backend as K

In [ ]:
def good_accuracy(y_true, y_pred):
    return K.mean(K.equal(K.round(y_pred[:, 0]), y_true[:, 0]), axis=0)
def blurry_accuracy(y_true, y_pred):
    return K.mean(K.equal(K.round(y_pred[:, 1]), y_true[:, 1]), axis=0)
def dark_accuracy(y_true, y_pred):
    return K.mean(K.equal(K.round(y_pred[:, 2]), y_true[:, 2]), axis=0)

In [ ]:
# def class_accuracy(y_true, y_pred):
#     return np.mean(np.equal(np.round(y_pred), y_true), axis=0)

# y_true = np.round(np.random.rand(16, 3))
# y_pred = np.random.rand(16, 3)
# class_accuracy(y_true, y_pred)

In [ ]:
model.compile(adam, 
              loss="binary_crossentropy", 
              metrics=["binary_accuracy", good_accuracy, blurry_accuracy, dark_accuracy])

train

In [ ]:
from keras.callbacks import LearningRateScheduler, ModelCheckpoint, Callback
import math

In [ ]:
def step_decay(epoch):
    drop = 0.5
    epochs_drop = 15.0
    lrate = initial_lrate * math.pow(drop,  
           math.floor((1+epoch)/epochs_drop))
    return lrate
lrate = LearningRateScheduler(step_decay)

In [ ]:
# checkpoint = ModelCheckpoint("/root/data/priority_queue/models/frames_{epoch:02d}.h5", 
#                              monitor='val_loss', 
#                              verbose=0, 
#                              save_best_only=False, 
#                              save_weights_only=False, 
#                              mode='min', 
#                              period=1)
class CheckpointSingleModel(Callback):
    def on_epoch_end(self, epoch, logs={}):
        single_model.save("/root/data/priority_queue/models/frames_{}.h5".format(epoch))
        return


In [ ]:
slowdown = 2

In [ ]:
history = model.fit_generator(generator = traingen, 
                              steps_per_epoch=(np.min([len(train_good), len(train_bad)])) // (batch_size*slowdown),
                              workers=10,
                              max_queue_size=20,
                              use_multiprocessing=False,
                              validation_data=valgen,
                              validation_steps = (np.min([len(val_good), len(val_bad)])) // (batch_size*slowdown),
                              callbacks=[lrate, CheckpointSingleModel()],
                              epochs=30*slowdown)

In [ ]:
h = history.history

In [ ]:
plt.plot(h["loss"])
plt.plot(h["val_loss"])
plt.show()

In [ ]:
np.max(h["val_binary_accuracy"])

In [ ]:
# plt.plot(h["good_accuracy"])
# plt.plot(h["blurry_accuracy"])
# plt.plot(h["dark_accuracy"])
plt.plot(h["val_good_accuracy"])
plt.plot(h["val_blurry_accuracy"])
plt.plot(h["val_dark_accuracy"])
plt.show()

In [ ]:
# single_model.save("/root/data/priority_queue/models/draft_frames_3.h5")

In [ ]:
np.argmax(h["val_binary_accuracy"])

In [ ]:
import json
stuff = {}
stuff["description"] = "binary classification good / bad for filtering"
stuff["input_size"] = (224, 224, 3)
stuff["output_size"] = (3,)
stuff["probability"] = {"is_good": 0, "is_dark":2, "is_blurry": 1}
stuff["model"] = "Mobilenet"
with open("/root/data/priority_queue/models/blom-kjeppevikholmen_filtration_2019-03-08.json", "w") as f:
    json.dump(stuff, f)